#### SpiceyPy Simulation Analysis example
This example shows the basic CITROS data analysis functions using the results of Cassini orbit simulation.

First of all, we should import all neccesary libs:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable, ALL
import json
from platform import python_version
from citros import CitrosDB

Now we can connect to the simulation database:

In [ ]:
citros = CitrosDB(simulation='simulation_spiceypy_cassini', batch='spiceypy_cassini')
citros.info().print()

As you can see in the output above, we've got some information about our simulation run (batch):
1. Size of data
2. Sid information. In case we have more then one sid - this will be shown here.
3. Number and a list of topics. First three topics are build-in system topics, and the last one is our simulation results. 

Let's go further:

In [ ]:
citros.sid([0]).info().print()

This shows us more detailed statistics about topics: time stamps, durations, number of messages etc.
Let's inspect our data topic:

In [ ]:
citros.topic('/spiceypy_cassini/state').sid(0).info().print()

Here we can see that our data topic contains 400 messages, and each one's type is a list. 
Let's print all raw data:

In [ ]:
print(citros.topic('/spiceypy_cassini/state').data())

To get all json-data columns for a specific topic, function data() is used. It returns pandas DataFrame. To get a specific columns, pass their labels in a list as an argument:

In [ ]:
print(citros.topic('/spiceypy_cassini/state').data(['data.data[0]','data.data[1]'])[['data.data[0]', 'data.data[1]']])

Now we can create some plots. Let's plot coordinates along one of the axes (supposing messages come once a second): 

In [ ]:
fig1, ax1 = plt.subplots()

citros.time_plot(ax1, 
                 topic_name = '/spiceypy_cassini/state', 
                 var_name = 'data.data[0]', 
                 time_step = 1, 
                 sids = [0], 
                 y_label = 'X coords', title_text = 'X coords vs. Time')

To create a 3D plot of our Cassini trajectory, we can use Matplotlib standart functions:

In [ ]:
df = citros.topic('/spiceypy_cassini/state')\
           .sid([0])\
           .data(['data.data[0]', 'data.data[1]', 'data.data[2]'])

fig, ax = citros.plot_3dgraph(df, 'data.data[0]', 'data.data[1]', 'data.data[2]', 
                 scale = False, title = 'Results', set_x_label='X', set_y_label='Y', set_z_label='Z',
                 legend = True, )

ax.set_box_aspect(aspect=None, zoom=0.9)
ax.view_init(10, 10, 0)

fig.tight_layout() 

Let's do some simple research. We know trajectory and flight time (from the params.yaml or from simulation setup), so we can calculate velocity:
1. Flight duration: Jun 20, 2004 - Dec 1, 2004 (14 169 600 seconds)
2. 400 messages (1 message per 35 422.5 sec)

First, we will do it using standart Python (and Numpy) syntax:

In [ ]:
# Getting data to Pandas DataFrame
data=citros.topic('/spiceypy_cassini/state').data(['data.data[0]','data.data[1]','data.data[2]'])[['data.data[0]', 'data.data[1]','data.data[2]']]
# Converting to nympy array
data=data.to_numpy()

# Going into loop to calculate velocity
vel=[]
for i in range(2,len(data)):
    vel.append(np.sqrt((data[i,0]-data[i-1,0])**2 + (data[i,1]-data[i-1,1])**2 + (data[i,2]-data[i-1,2])**2))
# 

We also need a range of dates between our simulation date limits to create a beautiful plot:

In [ ]:
from datetime import datetime, timedelta

def generate_fixed_length_date_range(start_date, end_date, length):
    delta = (end_date - start_date) / (length - 1)
    date_range = [start_date + i * delta for i in range(length)]
    return date_range

# Define the start and end dates
start_date = datetime(2004, 6, 20)
end_date = datetime(2004, 12, 1)

# Define the desired fixed length
length = len(vel)

# Generate the fixed length date range
date_range = generate_fixed_length_date_range(start_date, end_date, length)

Finally, we are ready to plot:

In [ ]:
plt.plot(date_range, vel)

We can do the same calculations using Pandas lib:

In [ ]:
df=citros.topic('/spiceypy_cassini/state').data(['data.data[0]','data.data[1]','data.data[2]'])[['data.data[0]', 'data.data[1]','data.data[2]']]
velocity = []
for i in range(1, len(df)-1):
    velocity.append(np.sqrt(((df.iloc[i+1]- df.iloc[i])**2).sum()))
plt.plot(date_range, velocity)

Or, in addition, the same thing but in more tricky way (check our Data Analysis documentation, it's amazing! https://citros.io/doc/docs_data_analysis ):

In [ ]:
F = np.sqrt(((df.iloc[0:-1] - df.iloc[1:].reset_index(drop = True))**2).sum(axis = 1))
plt.plot(date_range, F.iloc[1:])